In [0]:
!pip install -r https://raw.githubusercontent.com/datamllab/automl-in-action-notebooks/master/requirements.txt


In [0]:
import tensorflow as tf

tf.random.set_seed(42)


## Load data


In [0]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


## Explore data


In [0]:
train_images.shape, test_images.shape

len(train_labels), len(test_labels)

train_labels, test_labels

import matplotlib.pyplot as plt

plt.figure()
plt.imshow(train_images[0])  # , cmap='gray'
plt.colorbar()
plt.title("Label is {label}".format(label=train_labels[0]))
plt.show()


## Data Preparation: scaling


In [0]:
train_images = train_images / 255.0
test_images = test_images / 255.0


In [0]:
%matplotlib inline


In [0]:
import matplotlib
import matplotlib.pyplot as plt

# plot first 20 images
n = 20
_, axes = plt.subplots(2, 10, figsize=(10, 2))
plt.tight_layout()
for i in range(n):
    row, col = i // 10, i % 10
    axes[row, col].set_axis_off()
    axes[row, col].imshow(
        train_images[
            i,
        ],
        cmap=plt.cm.binary,
        interpolation="nearest",
    )  # plt.cm.gray_r
    axes[row, col].set_title("Label: %i" % train_labels[i])


## Build MLP


In [0]:
from tensorflow import keras
from tensorflow.keras import layers

mlp_model = keras.Sequential(
    [
        keras.layers.Flatten(input_shape=train_images.shape[1:]),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ]
)
mlp_model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

mlp_model.summary()

mlp_model.fit(train_images, train_labels, epochs=5, batch_size=64, verbose=1)

test_loss, test_acc = mlp_model.evaluate(test_images, test_labels, verbose=0)
test_acc


## Build CNN


In [0]:

def build_cnn():
    model = keras.Sequential(
        [
            keras.layers.Conv2D(
                32, (3, 3), activation="relu", input_shape=train_images.shape[1:] + (1,)
            ),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )
    return model


cnn_model = build_cnn()

cnn_model.summary()

train_images_4d = train_images[..., tf.newaxis]
test_images_4d = test_images[..., tf.newaxis]
train_images_4d.shape, test_images_4d.shape

cnn_model.fit(train_images_4d, train_labels, epochs=5, batch_size=64, verbose=1)

test_loss, test_acc = cnn_model.evaluate(test_images_4d, test_labels, verbose=0)
test_acc


## Make predictions


In [0]:
test_predictions = cnn_model.predict(test_images_4d)
test_predictions[:5]

import numpy as np

np.argmax(test_predictions[0])

test_labels[0]
